# Assignment

In [ ]:
# Import 

import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *
from tqdm.notebook import tqdm

size_board = 4

In [ ]:
# Initialize the game environment
env=Chess_Env(size_board)

In [ ]:
# Initialize random seed for reproducibility
np.random.seed(41) # 41, 42, 43, 44, 45

# Initialize neural network
S,X,allowed_a=env.Initialise_game()

N_a = np.shape(allowed_a)[0] # Output size
N_in = np.shape(X)[0] # Input size
N_h = 200 # Hidden neurons

W1 = np.random.uniform(0,1,(N_h, N_in))
W2 = np.random.uniform(0,1,(N_a, N_h))
W1 = np.divide(W1,np.matlib.repmat(np.sum(W1,1)[:,None],1,N_in))
W2 = np.divide(W2,np.matlib.repmat(np.sum(W2,1)[:,None],1,N_h))

bias_W1 = np.zeros((N_h,))
bias_W2 = np.zeros((N_a,))

# Activate RMSprop
is_RMSprop = False
if is_RMSprop:
    squared_gradients = np.zeros(4)

# Hyperparamters
epsilon_0 = 0.2
beta = 0.00005 # epsilon decay
gamma = 0.85 # discount factor
eta = 0.0035 # learning rate

N_episodes = 10000 # number of episodes

R_save = np.zeros(N_episodes)
N_moves_save = np.zeros(N_episodes)

# Forward pass through neural network to compute Q-values
def predict(x0, W1, W2, bias_W1, bias_W2):
    h1 = np.dot(W1, x0) + bias_W1
    x1 = 1 / (1 + np.exp(-h1))
    h2 = np.dot(W2, x1) + bias_W2
    x2 = 1 / (1 + np.exp(-h2))
    return x1, x2

# Backpropagate error through neural network
def backprop(q, q_err, x0, x1, W1, W2, bias_W1, bias_W2):
    delta2 = q*(1-q) * q_err
    dW2 = np.outer(delta2, x1)
    delta1 = x1*(1-x1) * np.dot(W2.T, delta2)
    dW1 = np.outer(delta1, x0)
    
    W1 += eta * dW1
    W2 += eta * dW2
    bias_W1 += eta * delta1
    bias_W2 += eta * delta2
    
    return W1, W2, bias_W1, bias_W2

# Backpropagate error through neural network using RMSprop
def RMSprop(q, q_err, x0, x1, W1, W2, bias_W1, bias_W2, squared_gradients):
    delta2 = q*(1-q) * q_err
    dW2 = np.outer(delta2, x1)
    delta1 = x1*(1-x1) * np.dot(W2.T, delta2)
    dW1 = np.outer(delta1, x0)
    
    rms_beta = 0.9
    dW1_squared = rms_beta * squared_gradients[0] + (1-rms_beta) * np.square(dW1) # weighted average of previous squared gradient
    dW2_squared = rms_beta * squared_gradients[1] + (1-rms_beta) * np.square(dW2) # and current squared gradient
    alpha1 = eta / (1e-6 + np.sqrt(dW1_squared))
    alpha2 = eta / (1e-6 + np.sqrt(dW2_squared))
    
    db1_squared = rms_beta * squared_gradients[2] + (1-rms_beta) * np.square(delta1)
    db2_squared = rms_beta * squared_gradients[3] + (1-rms_beta) * np.square(delta2)
    alpha1b = eta / (1e-6 + np.sqrt(db1_squared))
    alpha2b = eta / (1e-6 + np.sqrt(db2_squared))
    
    W1 += alpha1 * dW1
    W2 += alpha2 * dW2
    bias_W1 += alpha1b * delta1
    bias_W2 += alpha2b * delta2
    
    new_squared_gradients = np.array([dW1_squared, dW2_squared, db1_squared, db2_squared], dtype=object)
    return W1, W2, bias_W1, bias_W2, new_squared_gradients

# Epsilon greedy policy
def policy(allowed_a, q, epsilon_f):
    possible_a = np.where(allowed_a==1)[0]
    q_a = q[possible_a]
    
    if np.random.random() < epsilon_f:
        return possible_a[np.random.randint(possible_a.size)]
    else:
        return possible_a[np.argmax(q_a)]

## Q-learning

In [ ]:
for n in tqdm(range(N_episodes)):

    epsilon_f = epsilon_0 / (1 + beta * n) # decay epsilon
    Done = 0
    i = 1 # number of moves
    R = 0
    
    _, X, allowed_a = env.Initialise_game()
    
    while not Done:
        
        # Make a move according to policy
        x1, q = predict(X, W1, W2, bias_W1, bias_W2)
        a = policy(allowed_a, q, epsilon_f)
        _, Xn, allowed_an, R, Done = env.OneStep(a)

        q_err = np.zeros(N_a)
        
        if Done:
            # Store statistics
            R_save[n] = R
            N_moves_save[n] = i
            
            q_err[a] = R - q[a]
            if is_RMSprop:
                W1, W2, bias_W1, bias_W2, squared_gradients = RMSprop(q, q_err, X, x1, W1, W2, bias_W1, bias_W2, squared_gradients)
            else:
                W1, W2, bias_W1, bias_W2 = backprop(q, q_err, X, x1, W1, W2, bias_W1, bias_W2)
            break
        
        # Predict next Q-values
        _, qn = predict(Xn, W1, W2, bias_W1, bias_W2)
        
        q_err[a] = (R + gamma * np.max(qn) - q[a])
        if is_RMSprop:
            W1, W2, bias_W1, bias_W2, squared_gradients  = RMSprop(q, q_err, X, x1, W1, W2, bias_W1, bias_W2, squared_gradients)
        else:
            W1, W2, bias_W1, bias_W2 = backprop(q, q_err, X, x1, W1, W2, bias_W1, bias_W2)
        
        # Update the current state, allowed action and move counter
        X = Xn
        allowed_a = allowed_an
        i += 1

## SARSA

In [ ]:
for n in tqdm(range(N_episodes)):

    epsilon_f = epsilon_0 / (1 + beta * n) # decay epsilon
    Done = 0
    i = 1 # number of moves
    R = 0
    
    _, X, allowed_a = env.Initialise_game() 
    x1, q = predict(X, W1, W2, bias_W1, bias_W2) 
    a = policy(allowed_a, q, epsilon_f) # choose a using policy
    
    while not Done:
        
        _, Xn, allowed_an, R, Done = env.OneStep(a) # take action a

        q_err = np.zeros(N_a)
        
        if Done:
            R_save[n] = R
            N_moves_save[n] = i
            
            q_err[a] = R - q[a]
            if is_RMSprop:
                W1, W2, bias_W1, bias_W2, squared_gradients = RMSprop(q, q_err, X, x1, W1, W2, bias_W1, bias_W2, squared_gradients)
            else:
                W1, W2, bias_W1, bias_W2 = backprop(q, q_err, X, x1, W1, W2, bias_W1, bias_W2)
            break
        
        _, qn = predict(Xn, W1, W2, bias_W1, bias_W2) # get next Q
        an = policy(allowed_an, qn, epsilon_f) # choose a' from S' using Q

        q_err[a] = R + gamma * qn[an] - q[a]
        if is_RMSprop:
            W1, W2, bias_W1, bias_W2, squared_gradients  = RMSprop(q, q_err, X, x1, W1, W2, bias_W1, bias_W2, squared_gradients)
        else:
            W1, W2, bias_W1, bias_W2 = backprop(q, q_err, X, x1, W1, W2, bias_W1, bias_W2)
        
        # Update the current state, actions and move counter
        X = Xn
        allowed_a = allowed_an
        a = an
        i += 1

In [ ]:
# Example code to store Reward and moves in a file
with open("R.npy", "wb") as f:
    np.save(f, np.array(R_save))

with open("moves.npy", "wb") as f:
    np.save(f, np.array(N_moves_save))

In [ ]:
def exp_moving_average(data,alpha):
    alpha_rev = 1-alpha

    scale = 1/alpha_rev
    n = data.shape[0]

    r = np.arange(n)
    scale_arr = scale**r
    offset = data[0]*alpha_rev**(r+1)
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

# Plot exp. moving average of reward and moves
fig, ax1 = plt.subplots()

ax1.set_xlabel('Game')
ax1.set_ylabel('Reward', color="red")
ax1.plot(exp_moving_average(R_save, 1/1000),color="red",label='Reward')
ax1.tick_params(axis='y', labelcolor="red")

ax2 = ax1.twinx()

ax2.set_ylabel('Moves',color="blue")
ax2.plot(exp_moving_average(N_moves_save, 1/1000),color="blue",label='Moves')
ax2.tick_params(axis='y', labelcolor="blue")

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)